# Induced gap and magnetic field

# TO-DO:
* Tune induced gap algo

In [ ]:
# 1. Standard library imports
import os.path

# 2. External package imports
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from toolz import partition_all

# 3. Internal imports
import funcs

hv.notebook_extension('matplotlib')
print(kwant.__version__)
import warnings
warnings.filterwarnings('ignore', r'Interpreting')

# Connect to ipyparallel

In [ ]:
from ipyparallel import Client
import hpc05
client = hpc05.Client(profile='pbs', timeout=60, extra_args='--timeout=1200')
# client = Client()
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print(len(dview))

%px import sys, os; sys.path.append(os.path.expanduser('~/Work/induced_gap_B_field/'))
%px import funcs

# Usage

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=7, mu_lead=7, c_tunnel=3/4, V=lambda x, y, z: 0, V_barrier=0, **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=200, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

syst = funcs.make_3d_wire(**syst_pars)

kwant.plot(syst);

In [ ]:
lead = syst.leads[0]

def get_cross_section(syst, pos, direction):
    coord = np.array([s.pos for s in syst.sites if s.pos[direction] == pos])
    cross_section = np.delete(coord, direction, 1)
    return cross_section

def get_densities(lead, k, params):

    xy = get_cross_section(lead, pos=0, direction=0)
    h, t = lead.cell_hamiltonian(params=params), lead.inter_cell_hopping(params=params)
    h_k = h + t * np.exp(1j * k) + t.T.conj() * np.exp(-1j * k)

    vals, vecs = np.linalg.eigh(h_k)
    indxs = np.argsort(np.abs(vals))
    vecs = vecs[:, indxs]
    vals = vals[indxs]
    
    norbs = funcs.lat_from_syst(lead).norbs
    densities = np.linalg.norm(vecs.reshape(-1, norbs, len(vecs)), axis=1)**2
    return xy, vals, densities.T

xy, energies, densities = get_densities(lead, 0.1, params)
wfs = [kwant.plotter.mask_interpolate(xy, density, oversampling=1)[0] for density in densities[:40]]

ims = {E: hv.Image(wf) for E, wf in zip(energies, wfs)}
hv.HoloMap(ims, kdims=[hv.Dimension('E', unit='meV')])

In [ ]:
%%opts Image [colorbar=True]

x = 20
xy = get_cross_section(syst, x, 0)

V = lambda x, y, z: 10 * z / syst_pars['r1']

potential = np.array([V(*s.pos) for s in syst.sites if s.pos[0]==x])

hv.Image(np.rot90(kwant.plotter.mask_interpolate(xy, potential, oversampling=1)[0]))

In [ ]:
%%opts Path [aspect='square']

lead = funcs.make_lead(10, 50, 70, 135, 90, True, True, 'circle').finalized()
params['B_x'] = 1
params['mu_lead'] = 10
params['B_y'] = 0
ks = np.linspace(-1, 1)
Es = funcs.bands(lead, params, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

lead = funcs.make_lead(10, 50, 70, 135, 0, False, True, 'circle').finalized()
Es1 = funcs.bands(lead, params, ks) +0.1
p2 = hv.Path((ks, Es1))[:, -100:100]

p1 * p2 + (p1*p2)[:, -2:2]

In [ ]:
%%time
gap = funcs.find_gap(lead, params)
print('The bandgap is {} meV'.format(gap))

In [ ]:
params = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=15, mu_lead=15, c_tunnel=3/4, V=lambda x,y,z:0, V_barrier=50, **funcs.constants.__dict__)

S = kwant.smatrix(syst, params=params)

In [ ]:
funcs.andreev_conductance(syst, params)

# Tuning the gap

In [ ]:
params = dict(c_tunnel=3/4, B_x=0, B_y=0, B_z=0, V_barrier=50, g=50,
              alpha=20, orbital=True, V='lambda x, y, z: 0',
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, coverage_angle=180, r1=35, r2=70,
                 shape='circle', with_shell=True, A_correction=True)

mus = np.linspace(5, 15, 51)
Deltas = np.linspace(0, 100, 101)

vals = funcs.named_product(Delta=Deltas, mu_lead=mus)

def func(val, syst_pars=syst_pars, params=params):
    import funcs
    import numpy as np
    lead = funcs.make_lead(**syst_pars).finalized()

    # Combine the fixed parameters `params` and the changing
    # parameters `val` to one dict and evaluate the string
    # lambda functions.
    params = funcs.parse_params(dict(**params, **val))

    # Create the Hamiltonian `ham` at k=0.
    h0 = lead.cell_hamiltonian(params=params)
    t0 = lead.inter_cell_hopping(params=params)
    ham = h0 + t0 + t0.conj().T

    # Find the energies.
    ev = np.linalg.eigvalsh(ham)
    
    # Return a combined dictionary with the results and input.
    return dict(E_min=np.abs(ev).min(), **val)

funcs.run_simulation(lview, func, vals, dict(**params, **syst_pars), 'data/gap_tuning.hdf', overwrite=True)

In [ ]:
# Plot the gaps and find the `Delta` that is needed for the required `Delta_induced`.
df = pd.read_hdf('data/gap_tuning.hdf')
Deltas = sorted(df.Delta.unique())
mus = sorted(df.mu_lead.unique())
gap = np.reshape(df.E_min.values, (len(Deltas), len(mus))) 
max_gap = np.max(np.min(gap, axis=1))
print("Max gap is {} meV".format(max_gap))

In [ ]:
hv.Curve((Deltas, gap.min(axis=1)), kdims=[r'$\Delta$'], vdims=[r'$E_{gap}$'],
         label=r"$E_{gap}$ over range of $\Delta$'s")

In [ ]:
Delta_ind = {delta: hv.Path((mus, evs), kdims=[r'$\mu$', ('E_gap', r'$E_{gap}$')]) 
      for evs, delta in zip(gap, Deltas)}

hm = hv.HoloMap(Delta_ind, kdims=[r'$\Delta$'])
(hm.select(E_gap=(0.3, .7)) * hv.HLine(0.4))

# adaptive

In [ ]:
import adaptive
from adaptive.learner import DataSaver, Learner1D, Learner2D, BalancingLearner
from adaptive.runner import SequentialExecutor
adaptive.notebook_extension()

In [ ]:
from functools import partial
from operator import itemgetter

params = dict(Delta=60, c_tunnel=3/4, V_barrier=50,
              V='lambda x, y, z: 0', mu=15,
             **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=2000, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

In [ ]:
vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           theta=[0, 90],
                           phi=[0, 90])

vals = [val for val in vals if (not (val['theta'] == 0 and val['phi'] != 0)) and
                               (not (val['g'] == 0 and val['orbital'] == False))]

def func(x, val, syst_pars=syst_pars, params=params):
    import funcs
    params['mu_lead'] = params['mu']
    params['B'], val['V_bias'] = x
    params['B_x'], params['B_y'], params['B_z'] = funcs.spherical_coords(
        params['B'], val['theta'], val['phi'])

    params = funcs.parse_params(dict(**params, **val))

    syst = funcs.make_3d_wire(**syst_pars)
    return dict(**funcs.andreev_conductance(syst, params, E=val['V_bias']), **val)

In [ ]:
learners = [Learner2D(partial(func, val=val), [(-1, 1), (0, 1)])
            for val in vals]

learner = DataSaver(BalancingLearner(learners), itemgetter('G_01'))

In [ ]:
runner = adaptive.Runner(learner, executor=client, 
                         goal=lambda l: l.loss() < 0.001,
                         log=True, shutdown_executor=False)

In [ ]:
import cloudpickle
with open('learners.pickle', 'rb') as f:
    learner = cloudpickle.load(f)

In [ ]:
%%opts Image {+framewise} [colorbar=True]
%%output size=200
vals2 = [funcs.add_direction(val) for val in vals]

plots = {tuple(val.values()): l.plot(400, 400, 0) for (l, val) in zip(learner.learners, vals2)}
hm = hv.HoloMap(plots, kdims=list(vals2[0].keys()))
hm

In [ ]:
(sum(l.n_real for l in learner.learners) / len(learner.learners))

In [ ]:
runner.task.cancel()

In [ ]:
learner = DataSaver(Learner1D(func, (-2, 2)), itemgetter('G_01'))

In [ ]:
runner = adaptive.Runner(learner, executor=client, 
                         goal=lambda l: l.loss() < 0.01,
                         log=True, shutdown_executor=False)

# ipyparallel

In [ ]:
# Make sure this folder is in your $PYTHONPATH
try:
    # If this can be imported, it assumes you are on the TU Delft network with access to cluster
    import hpc05
    client = hpc05.Client(profile='pbs', timeout=60, extra_args='--timeout=1200')
    print("Connected to hpc05")
except ImportError:
    from ipyparallel import Client
    client = Client()
    print("Connected to local engines")

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print(len(dview))

%px import sys, os; sys.path.append(os.path.expanduser('~/Work/induced_gap_B_field/'))
%px import funcs

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/200px-3D_Spherical.svg.png)

In [ ]:
params = dict(Delta=60, c_tunnel=3/4, **funcs.constants.__dict__)

syst_pars = dict(a=10, onsite_disorder=False,
                 L=2000, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)


vals = funcs.named_product(B=np.linspace(0, 2, 51),
                           theta=[0, 90],
                           phi=[0, 90],
                           V_bias=np.linspace(-0.25, 0.25, 51),
                           V_barrier=[50],
                           g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           mu=[5, 10, 15, 20],
                           V=['lambda x, y, z: 0',
                              'lambda x, y, z: 10 * z / 50',
                              'lambda x, y, z: -10 * z / 50'],
                           angle=[0, 45, 90])

vals = [val for val in vals if (not (val['theta'] == 0 and val['phi'] != 0)) and
                               (not (val['g'] == 0 and val['orbital'] == False))]


def func(val, syst_pars=syst_pars, params=params):
    import funcs
    val['mu_lead'] = val['mu']
    val['B_x'], val['B_y'], val['B_z'] = funcs.spherical_coords(val['B'],
                                                                val['theta'],
                                                                val['phi'])

    params = funcs.parse_params(dict(**params, **val))

    # Remove parameters from `params` that belong in `syst_pars`.
    for x in ['angle']:
        syst_pars[x] = params.pop(x)

    syst = funcs.make_3d_wire(**syst_pars)
    return dict(**funcs.andreev_conductance(syst, params, E=val['V_bias']), **val)

fname = "tmp/test_conductance_{}_with_V.hdf"
funcs.run_simulation(lview, func, vals, dict(**params, **syst_pars), fname, 200)

In [ ]:
syst_pars = dict(a=10, angle=45, coverage_angle=180, r1=35, r2=70, shape='circle', with_shell=True)

def gap(val, syst_pars=syst_pars, params=params):
    import funcs
    val['B_x'], val['B_y'], val['B_z'] = val.pop('B')
    params = funcs.parse_params(dict(**params, **val))
    lead = funcs.make_lead(**syst_pars)
    return dict(E_gap=funcs.find_gap(lead, params), **val)

fname = "tmp/gaps_{}.hdf"
funcs.run_simulation(lview, gap, vals, dict(**params, **syst_pars), fname, 200)